## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

pd.options.mode.chained_assignment = None  # default='warn'

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_2000Cases_NEq/'
# TestDir    = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_Test/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

NVarsRed           = 7

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 20.

FixedMinVal        = 1.e-12

In [3]:
Vars = []
for iVars in range(1,NVarsRed+1):
    Vars.append('PC_'+str(iVars))

Vars0 = []
for iVars in range(1,NVarsRed+1):
    Vars0.append('PC0_'+str(iVars))
    
SVars = []
for iVars in range(1,NVarsRed+1):
    SVars.append('SPC_'+str(iVars))
    

FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
NICs         = len(ICVecs)


P0Vec        = []
EqRatio0Vec  = []
T0Vec        = []
for iC in range(NICs):
    Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
    P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
    EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
    T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(NICs):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)
for iC in range(NICs):
    FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
    DataNew          = pd.read_csv(FileName, header=0)
    Nts              = len(DataNew)
    DataTemp         = DataNew.iloc[[0]][Vars]#DataTemp.iloc[1][Vars]
    DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
    DataTemp.reset_index(drop=True, inplace=True)
    DataNew[Vars0]   = DataTemp
    DataTemp['t']    = DataNew['t']
    DataNew['t']     = DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy()
    if (iC==0):
        Data        = DataNew
    else:
        Data        = pd.concat([Data, DataNew], axis=0)
Data.reset_index(drop=True, inplace=True)

In [4]:
Data

,t,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC0_1,PC0_2,PC0_3,PC0_4,PC0_5,PC0_6,PC0_7
0,0.000000e+00,-55.389973,-0.861183,0.071574,-0.344382,0.090233,-0.198915,-0.007655,-55.389973,-0.861183,0.071574,-0.344382,0.090233,-0.198915,-0.007655
1,5.693455e-14,-55.389449,-0.861173,0.071577,-0.344378,0.090231,-0.198916,-0.007657,-55.389973,-0.861183,0.071574,-0.344382,0.090233,-0.198915,-0.007655
2,1.171107e-13,-55.388896,-0.861162,0.071580,-0.344374,0.090230,-0.198916,-0.007659,-55.389973,-0.861183,0.071574,-0.344382,0.090233,-0.198915,-0.007655
3,1.807128e-13,-55.388310,-0.861151,0.071583,-0.344370,0.090228,-0.198917,-0.007662,-55.389973,-0.861183,0.071574,-0.344382,0.090233,-0.198915,-0.007655
4,2.479362e-13,-55.387692,-0.861139,0.071587,-0.344365,0.090226,-0.198917,-0.007665,-55.389973,-0.861183,0.071574,-0.344382,0.090233,-0.198915,-0.007655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,8.013230e-01,22.643171,0.165263,-0.538848,-0.132641,0.022487,0.112494,0.020489,-34.008478,0.472422,-0.173487,-0.247051,0.265047,0.099412,0.348454
24996,8.469460e-01,22.643171,0.165263,-0.538848,-0.132641,0.022487,0.112494,0.020489,-34.008478,0.472422,-0.173487,-0.247051,0.265047,0.099412,0.348454
24997,8.951665e-01,22.643171,0.165263,-0.538848,-0.132641,0.022487,0.112494,0.020489,-34.008478,0.472422,-0.173487,-0.247051,0.265047,0.099412,0.348454
24998,9.461324e-01,22.643171,0.165263,-0.538848,-0.132641,0.022487,0.112494,0.020489,-34.008478,0.472422,-0.173487,-0.247051,0.265047,0.099412,0.348454


In [5]:
# ### Retrieving Data
    
# FileName             = OutputDir + '/Orig/train/ext/t.csv'
# Data                 = pd.read_csv(FileName, header=None)
# tVec                 = Data.to_numpy()[:,0]
# tVec                 = [tVec[i] if tVec[i] > FixedMinVal else FixedMinVal for i in range(len(tVec))]

# FileName             = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv'
# Data                 = pd.read_csv(FileName, header=0)
# Data[Vars0]          = Data0
# Data['t']            = tVec
# Data['log(t)']       = np.log(tVec)
# Data['log10(t)']     = np.log10(tVec)
# #Data['Phi0']         = EqRatio0Vec
# #Data['T0']           = T0Vec
# #Data['P0']           = P0Vec
# yMat_pca             = Data[Vars].to_numpy()

# # FileName             = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PCSource.csv'
# # DataS                = pd.read_csv(FileName, header=0)
# # DataS[Vars0]         = Data0
# # DataS['t']           = tVec
# # DataS['log(t)']      = np.log(tVec)
# # DataS['log10(t)']    = np.log10(tVec)
# # DataS['Phi0']        = EqRatio0Vec
# # DataS['T0']          = T0Vec
# # DataS['P0']          = P0Vec
# # ySource_pca          = DataS[SVars].to_numpy()


# ### Writing Input
InputVar     = ['t'] + Vars0
# DataInput    = Data[InputVar]
# DataInput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/ext/Input.csv', index=False)
# tVec         = DataInput.t.to_numpy()


# ### Writing PCs
# Data.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/ext/Output.csv', index=False)


# ### Normalizing PCs' Source Terms
# yMax         = ySource_pca.max(0)
# yMin         = ySource_pca.min(0)
# yRange       = (yMax-yMin)
# yScale       = np.concatenate([yMin[...,np.newaxis], yMax[...,np.newaxis]], axis=1)
# np.savetxt(OutputDir+'/'+str(NVarsRed)+'PC/train/ext/MinMax.csv', yScale, header='Min,Max', comments='', delimiter=',')

# ySourcee_pca = (ySource_pca - yMin)/yRange
# #ySourcee_pca = (ySource_pca - yMin)/yRange[0]

# for iPC in range(NVarsRed):
#     DataS['SPC_'+str(iPC+1)+'_Scaled'] = ySourcee_pca[:,iPC]
# DataS.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/ext/dOutput.csv', index=False)

In [6]:
# # Plotting Sources vs States

# fig  = plt.figure(figsize=(16,12))
# ax   = plt.axes(projection='3d')
# ax.plot3D(yMat_pca[:,0], T0Vec, ySource_pca[:,0], 'ok',  linewidth=2)
# ax.set_xlabel(r'$PC_1$')
# ax.set_ylabel(r'$PC_2$')
# ax.set_zlabel(r'$dPC_1/dt$')

# fig  = plt.figure(figsize=(16,12))
# ax   = plt.axes(projection='3d')
# ax.plot3D(yMat_pca[:,0], T0Vec, ySource_pca[:,1], 'ok',  linewidth=2)
# ax.set_xlabel(r'$PC_1$')
# ax.set_ylabel(r'$PC_2$')
# ax.set_zlabel(r'$dPC_2/dt$')

# fig  = plt.figure(figsize=(16,12))
# ax   = plt.axes(projection='3d')
# ax.plot3D(yMat_pca[:,0], T0Vec, ySource_pca[:,2], 'ok',  linewidth=2)
# ax.set_xlabel(r'$PC_1$')
# ax.set_ylabel(r'$PC_2$')
# ax.set_zlabel(r'$dPC_3/dt$')

In [7]:
# # Plotting Sources vs States

# fig  = plt.figure(figsize=(16,12))
# ax   = plt.axes(projection='3d')
# ax.plot3D( yMat_pca[:,0], yMat_pca[:,1], ySourcee_pca[:,0], 'ok',  linewidth=2)
# ax.set_xlabel(r'$PC_1$')
# ax.set_ylabel(r'$PC_2$')
# ax.set_zlabel(r'$dPC_1/dt$')

# fig  = plt.figure(figsize=(16,12))
# ax   = plt.axes(projection='3d')
# ax.plot3D( yMat_pca[:,0], yMat_pca[:,1], ySourcee_pca[:,1], 'ok',  linewidth=2)
# ax.set_xlabel(r'$PC_1$')
# ax.set_ylabel(r'$PC_2$')
# ax.set_zlabel(r'$dPC_2/dt$')

# fig  = plt.figure(figsize=(16,12))
# ax   = plt.axes(projection='3d')
# ax.plot3D( yMat_pca[:,0], yMat_pca[:,1], ySourcee_pca[:,2], 'ok',  linewidth=2)
# ax.set_xlabel(r'$PC_1$')
# ax.set_ylabel(r'$PC_2$')
# ax.set_zlabel(r'$dPC_3/dt$')

In [8]:
# ### Checking Data

# from numpy import diff

# iPC = 0

# dydt = diff(yMat_pca[:,iPC])/diff(tVec)

# fig = plt.figure(figsize=(16,12))
# plt.plot(tVec, ySource_pca[:,iPC], 'ko')
# plt.plot(tVec[0:-1], dydt, 'ro')
# plt.xscale('log')

In [9]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/')
except:
    pass

In [10]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [11]:
data_id    = 'res'

DataNoZero           = Data[Data['t'] > FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput        = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

KeyError: "['log10(t)', 'log(t)'] not in index"

In [ ]:
data_id = 'ics'

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

Data_ics                     = Data[Data['t'] == FixedMinVal]
n_points_ics                 = len(Data_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=41)

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = Data_ics[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [ ]:
data_id = 'scs'

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

Mask                         = [i for i in range(1,len(Data)-1) if Data.iloc[i-51]['t'] == FixedMinVal]
Data_ics                     = Data.iloc[Mask]
n_points_ics                 = len(Data_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=41)

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = Data_ics[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [ ]:
data_id = 'fcs'

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

Mask                         = [i for i in range(-1,len(Data)-1) if Data.iloc[i+1]['t'] == FixedMinVal]
Data_ics                     = Data.iloc[Mask]
n_points_ics                 = len(Data_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=43)

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = Data_ics[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [ ]:
# ### Retrieving Data

# Vars = []
# for iVars in range(1,NVarsRed+1):
#     Vars.append('PC_'+str(iVars))

# SVars = []
# for iVars in range(1,NVarsRed+1):
#     SVars.append('SPC_'+str(iVars))

# FileName     = TestDir + '/orig_data/RestVecTot.csv'
# Data         = pd.read_csv(FileName, header=None)
# RestVecTot   = np.log10(Data.to_numpy())
    
# # FileName     = TestDir+'/orig_data/t.csv'
# # Data         = pd.read_csv(FileName, header=None)
# # tVec         = Data.to_numpy()[:,0]

# # FileName     = TestDir + '/orig_data/yCleaned.csv'
# # yMatOrig     = pd.read_csv(FileName, header=0)
# # VarNames     = list(yMatOrig.columns)
# # yMat         = yMatOrig.to_numpy()

# FileName             = TestDir+'/'+str(NVarsRed)+'PC/PC.csv'
# Data                 = pd.read_csv(FileName, header=0)
# Data['log10(t)']     = np.log10(Data.t.to_numpy()+1.e-15)
# Data['log10(Rest)']  = RestVecTot
# Data['Rest']         = 10.**RestVecTot
# yMat_pca             = Data[Vars].to_numpy()

# FileName             = TestDir+'/'+str(NVarsRed)+'PC/PCSource.csv'
# DataS                = pd.read_csv(FileName, header=0)
# DataS['log10(t)']    = np.log10(Data.t.to_numpy()+1.e-15)
# DataS['log10(Rest)'] = RestVecTot
# DataS['Rest']        = 10.**RestVecTot
# ySource_pca          = DataS[SVars].to_numpy()


# ### Writing Input
#InputVar     = ['t', 'log10(t)', 'Rest', 'log10(Rest)']
# DataInput    = Data[InputVar]
# DataInput.to_csv(TestDir+'/'+str(NVarsRed)+'PC/Input.csv', index=False)
# tVec         = DataInput.t.to_numpy()


# ### Writing PCs
# Data.to_csv(TestDir+'/'+str(NVarsRed)+'PC/Output.csv', index=False)


# ### Normalizing PCs' Source Terms
# ySourcee_pca = (ySource_pca - yMin)/yRange
# #ySourcee_pca = (ySource_pca - yMin)/yRange[0]

# for iPC in range(NVarsRed):
#     DataS['SPC_'+str(iPC+1)+'_Scaled'] = ySourcee_pca[:,iPC]
# DataS.to_csv(TestDir+'/'+str(NVarsRed)+'PC/dOutput.csv', index=False)